In [1087]:
class Nodo():
#constructor
  def __init__(self,dato,nivel,padre,coste):
    self.dato=dato
    self.nivel=nivel
    self.padre=padre
    self.coste=coste

 #obtener datos
  def obtener_datos(self):
    return self.dato

  #asignar dato
  def asignar_datos(self,dato):
    self.dato=dato

  def obtener_coste(self):
    return self.coste

  def asignar_coste(self,coste):
    self.coste=coste

  #asignar hijos
  def asignar_hijos(self,hijos):
    self.hijos=hijos
    if self.hijos!=None:  # si en la lista hay hijos entonces ingresa al for(ciclo)
      for hijito in self.hijos:
          hijito.padre=self

  #obtener hijos
  def obtener_hijos(self):
    return self.hijos

  #obtener padre
  def obtener_padre(self):
    return self.padre

  #asignar padre
  def asignar_padre(self,padre):
    self.padre=padre

  def igual(self,nodo):
    return np.array_equal(self.obtener_datos(), nodo.obtener_datos())

  def en_lista(self, lista_nodos):
      en_la_lista=False
      for n in lista_nodos:
          if self.igual(n):
            en_la_lista=True
      return en_la_lista



In [1088]:
def buscar_nodo_elegido(lista_nodos_hijos): #de todos los nodos  o matrices hijos resultantes solo el que tiene mayor costo sera el nodo elegido
    menor_coste = float('inf')  # Inicializar con infinito para asegurarse de que cualquier coste sea menor
    nodo_eficiente = None

    for item in lista_nodos_hijos:
        lista1 = encontrar_cero(item.obtener_datos())
        if item.obtener_coste() < menor_coste:
            menor_coste = item.obtener_coste()
            nodo_eficiente = item
        elif item.obtener_coste() == menor_coste:
            x1 = lista1[0]  # fila
            y1 = lista1[1]  # columna
            if x1 + y1 >= 2 and 0 < x1 < 3 and 0 < y1 < 3:
                # Si el nodo actual cumple con las condiciones de estar en el centro se actualiza como nodo actual
                nodo_eficiente = item

    return nodo_eficiente, lista1

In [1089]:
def encontrar_cero(nodo):
  fila=0
  columna=0
  for i in range(len(nodo)):
      for j in range(len(nodo[0])):
          if nodo[i][j]==0:
               fila=i
               columna=j
               break
  return [fila,columna]

In [1090]:
def nodoresultante3x3(nodo,posicionx,posiciony,sumarorestarX,sumarorestarY): #se intercambia el cero a otra celda
  nuevo_x = posicionx + sumarorestarX
  nuevo_y = posiciony + sumarorestarY
  # Verificar que los nuevos índices estén dentro de los límites de la matriz
  if 0 <= nuevo_x < len(nodo) and 0 <= nuevo_y < len(nodo[0]):
      # Intercambiar el cero a la nueva posición
      nodo[posicionx][posiciony] = nodo[nuevo_x][nuevo_y]
      nodo[nuevo_x][nuevo_y] = 0
      return nodo
  else:
      # Devolver None si los nuevos índices están fuera de los límites
      return None

In [1091]:
#con este cambio ya se calcula la heuristica como lo enseño la profe
def costodelnodo(estado_final,nodo): # obtenemos los numeros de celdas parecidas a la matriz resultantew
   costo=0
   for i in range(len(estado_final)):
    for j in range(len(estado_final[0])):
        if estado_final[i][j] != nodo.obtener_datos()[i][j]:
            costo+=1  # contrar cantidad de celdas que se parecen al nodo final(objetivo)
   return costo + nodo.nivel

In [1092]:
import copy
def busqueda_heuristica(e_inicial,e_final):
  # print('estado inicial desde heuristica:',e_inicial)
  exito=False #variable  sirve para salir del ciclo
  nodos_frontera=[] #cola donde se añadiran los nodos
  nodos_visitados=[] #lista para guardar los nodos que ya pasaron por el analisis(para enviar a lista_nodos)
  nodo_elegido=[]
  # Creación del primer nodo inicial o un objeto de tipo Nodo
  NodoInicial = Nodo(e_inicial,0,None,0) #nodo inicial
  # print('estado inicial Nodo desde heuristica:', NodoInicial.dato)
  nodos_frontera.append(NodoInicial) #agrego el nodo inicial a la cola
  comparaciones=0
  textoError="" #solo cuando hay error envia mensaje
  while len(nodos_frontera)!=0  and (not exito): #siempre y cuando exito = false y la cantidad de nodos frontera sea diferente a 0
    #nodo es el Nodo padre inicialmente y nodos_frontera llegaria a ser una cola porque saca el primero que estuvo en la cola y asi sucesivamente
    nodo = nodos_frontera.pop(0) #(asignamos el primer nodo que esta en cola en nodos frontera a la variable nodo)
    nodos_visitados.append(nodo) #tambien se añade a nodos ya visitados

    if np.array_equal(nodo.obtener_datos(), e_final): #si el nodo es igual al nodo resultado al que se quiere llegar se termina el ciclo y se imprime
      print('nodo final: ', nodo.obtener_datos())
      exito=True
      return nodo
    else:
      comparaciones = comparaciones + 1  #se incrementan las comparaciones
      aux = nodo.obtener_datos() #guardar la matriz
      fila=0
      columna=0
      x=len(aux) #hallar la cantidad de filas de aux
      y=len(aux[0]) #hallar la cantidad de columnas
      if aux is None:
        textoError="Error: El nodo tiene datos None."

      for i in range(x):
        nodos_hijos=[]
        #hallamos el centro de la matriz
        if fila == columna and (fila + columna) == 2:
           mataux = nodoresultante3x3(copy.deepcopy(aux),fila,columna,0,-1)#para mover el cero a la izquierda
           nodo_izquierdo=Nodo(mataux,nodo.nivel + 1,nodo,0)
           nodo_izquierdo.asignar_coste( costodelnodo(e_final,nodo_izquierdo) )

           mataux = nodoresultante3x3(copy.deepcopy(aux),fila,columna,-1,0)#para mover el cero hacia arriba
           nodo_arriba=Nodo(mataux,nodo.nivel + 1,nodo,0)
           nodo_arriba.asignar_coste( costodelnodo(e_final,nodo_arriba) )

           mataux = nodoresultante3x3(copy.deepcopy(aux),fila,columna,0,1)#para mover el cero hacia la derecha
           nodo_derecho=Nodo(mataux,nodo.nivel + 1,nodo,0)
           nodo_derecho.asignar_coste( costodelnodo(e_final,nodo_derecho) )

           mataux = nodoresultante3x3(copy.deepcopy(aux),fila,columna,1,0)#para mover el cero hacia abajo
           nodo_abajo=Nodo(mataux,nodo.nivel + 1,nodo,0)
           nodo_abajo.asignar_coste( costodelnodo(e_final,nodo_abajo) )

           if nodo_izquierdo.en_lista(nodos_visitados)  == False:  #si es true es porque el nodo ya ha sido recorrido, si es false aun no esta en la lista y se guarda en ella
              nodos_hijos.append(nodo_izquierdo)
           if nodo_arriba.en_lista(nodos_visitados) == False:
              nodos_hijos.append(nodo_arriba)
           if nodo_derecho.en_lista(nodos_visitados) == False:
              nodos_hijos.append(nodo_derecho)
           if nodo_abajo.en_lista(nodos_visitados) == False:
              nodos_hijos.append(nodo_abajo)
          #se busca el nodo con el mayor número de items en común
           nodo_elegido, lista1=buscar_nodo_elegido(nodos_hijos)
           nodos_frontera.append(nodo_elegido)#guardamos ese nodo elegido
           fila = lista1[0]
           columna = lista1[1]
        elif fila==1 or columna==1:
          #si el cero esta en uno de los centros laterales
          if fila==1:
             mataux=nodoresultante3x3(copy.deepcopy(aux),fila,columna,-1,0)#mover el cero hacia arriba
             nodo_arriba=Nodo(mataux,nodo.nivel + 1,nodo,0)
             nodo_arriba.asignar_coste( costodelnodo(e_final,nodo_arriba) )

             mataux=nodoresultante3x3(copy.deepcopy(aux),fila,columna,1,0)#mover el cero hacia abajo
             nodo_abajo=Nodo(mataux,nodo.nivel + 1,nodo,0)
             nodo_abajo.asignar_coste( costodelnodo(e_final,nodo_abajo) )

             if fila > columna:
                mataux=nodoresultante3x3(copy.deepcopy(aux),fila,columna,0,1)#mover el cero hacia el centro por la derecha
                nodo_centro=Nodo(mataux,nodo.nivel + 1,nodo,0)
                nodo_centro.asignar_coste( costodelnodo(e_final,nodo_centro) )
             else:
                mataux=nodoresultante3x3(copy.deepcopy(aux),fila,columna,0,-1)#mover el cero hacia el centro por la izquierda
                nodo_centro=Nodo(mataux,nodo.nivel + 1,nodo,0)
                nodo_centro.asignar_coste( costodelnodo(e_final,nodo_centro) )

             if nodo_arriba.en_lista(nodos_visitados) == False:
                 nodos_hijos.append(nodo_arriba)
             if nodo_abajo.en_lista(nodos_visitados) == False:
                 nodos_hijos.append(nodo_abajo)
             if nodo_centro.en_lista(nodos_visitados) == False:
                 nodos_hijos.append(nodo_centro)
             nodo_elegido, lista1=buscar_nodo_elegido(nodos_hijos)
             nodos_frontera.append(nodo_elegido)#guardamos ese nodo elegido
             fila = lista1[0]
             columna = lista1[1]
          else:
            mataux = nodoresultante3x3(copy.deepcopy(aux),fila,columna,0,-1)
            nodo_izquierda=Nodo(mataux,nodo.nivel + 1,nodo,0)
            nodo_izquierda.asignar_coste( costodelnodo(e_final,nodo_izquierda) )

            mataux = nodoresultante3x3(copy.deepcopy(aux),fila,columna,0,1)
            nodo_derecha=Nodo(mataux,nodo.nivel + 1,nodo,0)
            nodo_derecha.asignar_coste( costodelnodo(e_final,nodo_derecha) )

            if columna>fila:
              mataux = nodoresultante3x3(copy.deepcopy(aux),fila,columna,1,0)
              nodo_centro=Nodo(mataux,nodo.nivel + 1,nodo,0)
              nodo_centro.asignar_coste( costodelnodo(e_final,nodo_centro) )
            else:
              mataux = nodoresultante3x3(copy.deepcopy(aux),fila,columna,-1,0)
              nodo_centro=Nodo(mataux,nodo.nivel + 1,nodo,0)
              nodo_centro.asignar_coste( costodelnodo(e_final,nodo_centro) )

            #si no estan en lista se añaden a la lista de nodos hijos
            if nodo_izquierda.en_lista(nodos_visitados) == False :
              nodos_hijos.append(nodo_izquierda)
            if nodo_centro.en_lista(nodos_visitados) == False :
              nodos_hijos.append(nodo_centro)
            if nodo_derecha.en_lista(nodos_visitados) == False :
              nodos_hijos.append(nodo_derecha)
            fila=2
            columna=0
          nodo_elegido, lista1=buscar_nodo_elegido(nodos_hijos)
          nodos_frontera.append(nodo_elegido)
          fila = lista1[0]
          columna = lista1[1]

        else:
            #si en una de las esquinas esta el cero
            if fila<=columna and (fila+columna)<len(aux) : # Caso cuando el cero esta en las esquinas superiores
              #esquinas
              mataux= nodoresultante3x3(copy.deepcopy(aux),fila,columna,1,0)#mover cero hacia abajo
              nodo_abajo=Nodo(mataux,nodo.nivel + 1,nodo,0)
              nodo_abajo.asignar_coste( costodelnodo(e_final,nodo_abajo) )
              fila +=1
              columna +=1
              if columna<=fila:
                mataux=nodoresultante3x3(copy.deepcopy(aux),fila,columna,0,1)#mover cero a la derecha
                nodo_lado=Nodo(mataux,nodo.nivel + 1,nodo,0)
                nodo_lado.asignar_coste( costodelnodo(e_final,nodo_lado) )
              else:
                mataux=nodoresultante3x3(copy.deepcopy(aux),fila,columna,0,-1)#mover cero  a la izquierda
                nodo_lado=Nodo(mataux,nodo.nivel + 1,nodo,0)
                nodo_lado.asignar_coste( costodelnodo(e_final,nodo_lado) )
              if nodo_abajo.en_lista(nodos_visitados) == False:
                  nodos_hijos.append(nodo_abajo)
              if nodo_lado.en_lista(nodos_visitados) == False:
                  nodos_hijos.append(nodo_lado)
              nodo_elegido, lista1=buscar_nodo_elegido(nodos_hijos)
              nodos_frontera.append(nodo_elegido)#guardamos ese nodo elegido
              fila = lista1[0]
              columna = lista1[1]
            else:# Caso cuando el cero esta en las esquinas inferiores
                 mataux = nodoresultante3x3(copy.deepcopy(aux),fila,columna,-1,0)#mover cero hacia arriba ( restamos una fila)
                 nodo_arriba=Nodo(mataux,nodo.nivel + 1,nodo,0)
                 nodo_arriba.asignar_coste( costodelnodo(e_final,nodo_arriba) )

                 if columna< fila:
                    mataux= nodoresultante3x3(copy.deepcopy(aux),fila,columna,0,1)#mover cero a la derecha (aumentamos una columna)
                    nodo_abajo=Nodo(mataux,nodo.nivel + 1,nodo,0)
                    nodo_abajo.asignar_coste( costodelnodo(e_final,nodo_abajo) )
                 else:
                    nodo_abajo= nodoresultante3x3(copy.deepcopy(aux),fila,columna,0,-1)#mover cero a la izquierda (restamos una columna)
                    nodo_abajo=Nodo(mataux,nodo.nivel + 1,nodo,0)
                    nodo_abajo.asignar_coste( costodelnodo(e_final,nodo_abajo) )

                 if nodo_arriba.en_lista(nodos_visitados) == False:
                     nodos_hijos.append(nodo_arriba)
                 if nodo_abajo.en_lista(nodos_visitados) == False:
                     nodos_hijos.append(nodo_abajo)
            nodo_elegido, lista1=buscar_nodo_elegido(nodos_hijos)
            nodos_frontera.append(nodo_elegido)
            fila = lista1[0]
            columna = lista1[1]
    return nodo_elegido




In [1093]:
import numpy as np

# estado_inicial = np.array([[1,2,3],[4,0,6],[7,5,8]])
estado_inicial = np.array([[1,6,7],[3,5,2],[8,0,4]]) # estaba probando con este otro estado
estado_final = np.array([[1,2,3],[4,5,6],[7,8,0]])

solucion = busqueda_heuristica(estado_inicial,estado_final)

# print("La solución es ->: ",solucion.obtener_datos()) #muestra la lista o el mensaje de error
nivel=0
datos_camino=[] #para hallar el camino que se tuvo que recorrer en toda la lista(esta lista guarda matrices)
nodo_aux=solucion
if(solucion!="Sin Nodos, No solucionado"):
  while(nodo_aux is not None and nodo_aux.obtener_padre() is not None):  #ciclo acaba cuando el nodo ya no tenga padre
      datos_camino.append(nodo_aux.obtener_datos())
      lista=nodo_aux.obtener_datos()
      nodo_aux=nodo_aux.obtener_padre()

  datos_camino.append(estado_inicial)
  datos_camino= list(reversed(datos_camino)) #para voltear la matriz
  print('datos_camino:', len(datos_camino))
  print("El camino es:")
  for i in range(len(datos_camino)): #obtenermos cantidad de filas
      for j in range( len(datos_camino[i])): #obtenemos cantidad de columnas
          print("\033[1;33m"+"[",datos_camino[i][j][0],",",datos_camino[i][j][1],",",datos_camino[i][j][2],"]"+'\033[')  #le damos un color amarillo

      print(" ")

datos_camino: 2
El camino es:
[ 1 , 6 , 7 ][
[ 3 , 5 , 2 ][
[ 8 , 0 , 4 ][
 
[ 1 , 6 , 7 ][
[ 3 , 5 , 0 ][
[ 8 , 0 , 2 ][
 
